### 1. Checkpointer概述
在实际应用中，我们经常会遇到这样的需求：
- 我们想知道Graph执行到某个节点时，State的值是什么？
- 我们想知道State在每一步是如何变化的？
- Graph执行出错了，我们想看看是哪一步的State出了问题？
- 我们想从某个历史状态重新开始，尝试不同的State值会带来什么结果？
为了解决这些问题，LangGraph提供了Checkpoints（检查点）机制。
Checkpoints就像是给Graph的State拍快照，它会在Graph执行的每个关键步骤自动保存State的完整信息。有了这些快照：
- 观察State的变化过程：查看Graph执行过程中每一步的State值，了解数据是如何流动和变化的。
- 回溯State的状态：当Graph执行出错时，可以回溯到出错前的checkpoint，查看当时的State的值，找出问题所在。
- 从历史State重新开始：从任意一个checkpoint的State重新开始执行，或者修改checkpoint的State后继续执行。
- 实验和优化：从同一个checkpoint尝试不同的State值，比较结果，优化Graph设计。
Checkpoints为我们提供了对State的观察和管理能力，让我们能够深入理解State的变化机制，并在实际应用中实现对State的灵活管理。
### 2.Checkpoints的创建
#### 2.1 什么是super-step
Graph的执行过程被划分为多个离散的super-step，每个super-step可以看作是图节点的一次迭代。
具体来说：
- 并行运行的节点属于同一个super-step：如果多个节点可以同时执行（比如它们都收到了消息，且彼此之间没有依赖），它们会在同一个super-step中并行运行
- 顺序运行的节点属于不同的super-step：如果节点A执行完后，节点B才执行，那么它们就不属于同一个super-step
#### 2.2 Checkpoints的创建
Checkpoint是Graph在每个super-step执行后自动保存的State快照，它由StateSnapshot对象表示，包含了State的值（values）、下一个要执行的节点（next）、配置信息（config）和元数据（metadata）等关键信息。